In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# NLP Model Training for SATIM FAQ Bot\n",
    "\n",
    "This notebook trains a natural language processing model for the SATIM call center FAQ bot using the scraped data."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "import os\n",
    "sys.path.append('../src')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import json\n",
    "import pickle\n",
    "from datetime import datetime\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from sklearn.feature_extraction.text import TfidfVectorizer\n",
    "from sklearn.metrics.pairwise import cosine_similarity\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import accuracy_score, classification_report\n",
    "import nltk\n",
    "from nltk.corpus import stopwords\n",
    "from nltk.tokenize import word_tokenize\n",
    "from nltk.stem import SnowballStemmer\n",
    "import re\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Download required NLTK data\n",
    "nltk.download('punkt', quiet=True)\n",
    "nltk.download('stopwords', quiet=True)\n",
    "nltk.download('punkt_tab', quiet=True)\n",
    "\n",
    "print(\"Setup complete!\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Scraped Data\n",
    "\n",
    "Let's load the FAQ data that we scraped in the previous notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load cleaned FAQ data\n",
    "df = pd.read_csv('../data/processed/satim_faqs_cleaned.csv')\n",
    "\n",
    "print(f\"Loaded {len(df)} FAQs\")\n",
    "print(f\"Categories: {list(df['category'].unique())}\")\n",
    "\n",
    "# Display basic statistics\n",
    "print(\"\\nDataset Overview:\")\n",
    "print(df.info())\n",
    "print(\"\\nFirst few rows:\")\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Text Preprocessing\n",
    "\n",
    "Let's create functions to preprocess French text for better NLP performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# French stopwords and stemmer\n",
    "french_stopwords = set(stopwords.words('french'))\n",
    "stemmer = SnowballStemmer('french')\n",
    "\n",
    "def preprocess_french_text(text):\n",
    "    \"\"\"\n",
    "    Preprocess French text for NLP\n",
    "    \"\"\"\n",
    "    if pd.isna(text) or not isinstance(text, str):\n",
    "        return \"\"\n",
    "    \n",
    "    # Convert to lowercase\n",
    "    text = text.lower()\n",
    "    \n",
    "    # Remove special characters but keep French accents\n",
    "    text = re.sub(r'[^\\w\\s\\-àâäçéèêëïîôöùûüÿ]', ' ', text)\n",
    "    \n",
    "    # Remove extra whitespace\n",
    "    text = re.sub(r'\\s+', ' ', text)\n",
    "    \n",
    "    # Tokenize\n",
    "    tokens = word_tokenize(text, language='french')\n",
    "    \n",
    "    # Remove stopwords and short words\n",
    "    tokens = [token for token in tokens if token not in french_stopwords and len(token) > 2]\n",
    "    \n",
    "    # Stem tokens\n",
    "    tokens = [stemmer.stem(token) for token in tokens]\n",
    "    \n",
    "    return ' '.join(tokens)\n",
    "\n",
    "# Test preprocessing function\n",
    "sample_text = \"Comment puis-je contacter le service client de SATIM pour résoudre mon problème?\"\n",
    "processed_text = preprocess_french_text(sample_text)\n",
    "print(f\"Original: {sample_text}\")\n",
    "print(f\"Processed: {processed_text}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Prepare Training Data\n",
    "\n",
    "Let's preprocess all our FAQ data and prepare it for model training."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Preprocess questions and answers\n",
    "print(\"Preprocessing text data...\")\n",
    "\n",
    "df['processed_question'] = df['question'].apply(preprocess_french_text)\n",
    "df['processed_answer'] = df['answer'].apply(preprocess_french_text)\n",
    "\n",
    "# Remove rows with empty processed text\n",
    "df = df[(df['processed_question'].str.len() > 0) & (df['processed_answer'].str.len() > 0)]\n",
    "\n",
    "print(f\"After preprocessing: {len(df)} FAQs\")\n",
    "\n",
    "# Show some examples\n",
    "print(\"\\nPreprocessing examples:\")\n",
    "for i in range(3):\n",
    "    print(f\"\\n--- Example {i+1} ---\")\n",
    "    print(f\"Original Q: {df.iloc[i]['question']}\")\n",
    "    print(f\"Processed Q: {df.iloc[i]['processed_question']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Build TF-IDF Vectorizer\n",
    "\n",
    "We'll use TF-IDF to convert text into numerical vectors for similarity matching."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create TF-IDF vectorizer optimized for French\n",
    "vectorizer = TfidfVectorizer(\n",
    "    max_features=5000,\n",
    "    ngram_range=(1, 2),  # Use unigrams and bigrams\n",
    "    min_df=2,  # Ignore terms that appear in less than 2 documents\n",
    "    max_df=0.8,  # Ignore terms that appear in more than 80% of documents\n",
    "    sublinear_tf=True\n",
    ")\n",
    "\n",
    "# Fit vectorizer on processed questions\n",
    "question_vectors = vectorizer.fit_transform(df['processed_question'])\n",
    "\n",
    "print(f\"TF-IDF matrix shape: {question_vectors.shape}\")\n",
    "print(f\"Vocabulary size: {len(vectorizer.vocabulary_)}\")\n",
    "\n",
    "# Show most important features\n",
    "feature_names = vectorizer.get_feature_names_out()\n",
    "print(f\"\\nSample features: {feature_names[:20]}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Create FAQ Similarity Model\n",
    "\n",
    "Let's create a model that can find the most similar FAQ for any given question."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "class FAQSimilarityModel:\n",
    "    def __init__(self, vectorizer, question_vectors, faq_data):\n",
    "        self.vectorizer = vectorizer\n",
    "        self.question_vectors = question_vectors\n",
    "        self.faq_data = faq_data.reset_index(drop=True)\n",
    "        \n",
    "    def find_best_match(self, query, top_k=3, min_similarity=0.1):\n",
    "        \"\"\"\n",
    "        Find the best matching FAQ for a given query\n",
    "        \"\"\"\n",
    "        # Preprocess query\n",
    "        processed_query = preprocess_french_text(query)\n",
    "        \n",
    "        if not processed_query:\n",
    "            return []\n",
    "        \n",
    "        # Vectorize query\n",
    "        query_vector = self.vectorizer.transform([processed_query])\n",
    "        \n",
    "        # Calculate similarities\n",
    "        similarities = cosine_similarity(query_vector, self.question_vectors).flatten()\n",
    "        \n",
    "        # Get top matches\n",
    "        top_indices = similarities.argsort()[-top_k:][::-1]\n",
    "        \n",
    "        results = []\n",
    "        for idx in top_indices:\n",
    "            similarity = similarities[idx]\n",
    "            if similarity >= min_similarity:\n",
    "                results.append({\n",
    "                    'question': self.faq_data.iloc[idx]['question'],\n",
    "                    'answer': self.faq_data.iloc[idx]['answer'],\n",
    "                    'category': self.faq_data.iloc[idx]['category'],\n",
    "                    'similarity': float(similarity),\n",
    "                    'confidence': self.calculate_confidence(similarity)\n",
    "                })\n",
    "        \n",
    "        return results\n",
    "    \n",
    "    def calculate_confidence(self, similarity):\n",
    "        \"\"\"\n",
    "        Convert similarity score to confidence level\n",
    "        \"\"\"\n",
    "        if similarity >= 0.8:\n",
    "            return 'high'\n",
    "        elif similarity >= 0.5:\n",
    "            return 'medium'\n",
    "        elif similarity >= 0.2:\n",
    "            return 'low'\n",
    "        else:\n",
    "            return 'very_low'\n",
    "\n",
    "# Create the model\n",
    "faq_model = FAQSimilarityModel(vectorizer, question_vectors, df)\n",
    "print(\"✓ FAQ Similarity Model created successfully\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Test the Model\n",
    "\n",
    "Let's test our model with various queries to see how well it performs."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test queries\n",
    "test_queries = [\n",
    "    \"Comment contacter SATIM?\",\n",
    "    \"Quels sont vos horaires d'ouverture?\",\n",
    "    \"Comment faire un paiement?\",\n",
    "    \"J'ai un problème technique\",\n",
    "    \"Où êtes-vous situés?\",\n",
    "    \"Comment créer un compte?\",\n",
    "    \"Problème avec ma carte\",\n",
    "    \"Tarifs et frais\"\n",
    "]\n",
    "\n",
    "print(\"Testing FAQ Model:\\n\")\n",
    "print(\"=\" * 80)\n",
    "\n",
    "for i, query in enumerate(test_queries, 1):\n",
    "    print(f\"\\n🔍 Test Query {i}: '{query}'\")\n",
    "    print(\"-\" * 50)\n",
    "    \n",
    "    results = faq_model.find_best_match(query, top_k=2)\n",
    "    \n",
    "    if results:\n",
    "        for j, result in enumerate(results, 1):\n",
    "            print(f\"\\n  Match {j} (Similarity: {result['similarity']:.3f}, Confidence: {result['confidence']})\")\n",
    "            print(f\"  Q: {result['question']}\")\n",
    "            print(f\"  A: {result['answer'][:150]}...\")\n",
    "            print(f\"  Category: {result['category']}\")\n",
    "    else:\n",
    "        print(\"  ❌ No suitable matches found\")\n",
    "\n",
    "print(\"\\n\" + \"=\" * 80)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Evaluate Model Performance\n",
    "\n",
    "Let's create a more systematic evaluation of our model's performance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create evaluation dataset by using existing questions\n",
    "# We'll modify them slightly to simulate real user queries\n",
    "\n",
    "def create_evaluation_queries(df, n_samples=50):\n",
    "    \"\"\"\n",
    "    Create evaluation queries by modifying existing questions\n",
    "    \"\"\"\n",
    "    eval_data = []\n",
    "    \n",
    "    # Sample questions\n",
    "    sample_df = df.sample(n=min(n_samples, len(df)), random_state=42)\n",
    "    \n",
    "    for idx, row in sample_df.iterrows():\n",
    "        original_question = row['question']\n",
    "        \n",
    "        # Create variations\n",
    "        variations = [\n",
    "            original_question,  # Exact match\n",
    "            original_question.replace('Comment', 'Comment puis-je'),\n",
    "            original_question.replace('?', ' s\\'il vous plaît?'),\n",
    "            original_question.lower(),\n",
    "            ' '.join(original_question.split()[:5]) + '?'  # Truncated\n",
    "        ]\n",
    "        \n",
    "        for variation in variations:\n",
    "            eval_data.append({\n",
    "                'query': variation,\n",
    "                'expected_idx': idx,\n",
    "                'expected_question': original_question,\n",
    "                'expected_category': row['category']\n",
    "            })\n",
    "    \n",
    "    return eval_data\n",
    "\n",
    "# Create evaluation dataset\n",
    "eval_data = create_evaluation_queries(df, n_samples=20)\n",
    "print(f\"Created {len(eval_data)} evaluation queries\")\n",
    "\n",
    "# Show some examples\n",
    "print(\"\\nEvaluation query examples:\")\n",
    "for i in range(3):\n",
    "    print(f\"Query: {eval_data[i]['query']}\")\n",
    "    print(f\"Expected: {eval_data[i]['expected_question']}\")\n",
    "    print()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Evaluate model performance\n",
    "def evaluate_model(model, eval_data):\n",
    "    \"\"\"\n",
    "    Evaluate the FAQ model performance\n",
    "    \"\"\"\n",
    "    results = {\n",
    "        'top1_accuracy': 0,\n",
    "        'top3_accuracy': 0,\n",
    "        'avg_similarity': 0,\n",
    "        'no_match_rate': 0,\n",
    "        'confidence_distribution': {'high': 0, 'medium': 0, 'low': 0, 'very_low': 0}\n",
    "    }\n",
    "    \n",
    "    top1_correct = 0\n",
    "    top3_correct = 0\n",
    "    total_similarity = 0\n",
    "    no_matches = 0\n",
    "    valid_queries = 0\n",
    "    \n",
    "    for eval_item in eval_data:\n",
    "        query = eval_item['query']\n",
    "        expected_question = eval_item['expected_question']\n",
    "        \n",
    "        matches = model.find_best_match(query, top_k=3, min_similarity=0.05)\n",
    "        \n",
    "        if matches:\n",
    "            valid_queries += 1\n",
    "            \n",
    "            # Check top-1 accuracy\n",
    "            if matches[0]['question'] == expected_question:\n",
    "                top1_correct += 1\n",
    "            \n",
    "            # Check top-3 accuracy\n",
    "            if any(match['question'] == expected_question for match in matches):\n",
    "                top3_correct += 1\n",
    "            \n",
    "            # Record similarity and confidence\n",
    "            total_similarity += matches[0]['similarity']\n",
    "            results['confidence_distribution'][matches[0]['confidence']] += 1\n",
    "        else:\n",
    "            no_matches += 1\n",
    "    \n",
    "    # Calculate metrics\n",
    "    total_queries = len(eval_data)\n",
    "    \n",
    "    if valid_queries > 0:\n",
    "        results['top1_accuracy'] = top1_correct / total_queries\n",
    "        results['top3_accuracy'] = top3_correct / total_queries\n",
    "        results['avg_similarity'] = total_similarity / valid_queries\n",
    "    \n",
    "    results['no_match_rate'] = no_matches / total_queries\n",
    "    \n",
    "    return results\n",
    "\n",
    "# Run evaluation\n",
    "print(\"Evaluating model performance...\")\n",
    "performance = evaluate_model(faq_model, eval_data)\n",
    "\n",
    "print(\"\\n📊 Model Performance Results:\")\n",
    "print(f\"Top-1 Accuracy: {performance['top1_accuracy']:.3f} ({performance['top1_accuracy']*100:.1f}%)\")\n",
    "print(f\"Top-3 Accuracy: {performance['top3_accuracy']:.3f} ({performance['top3_accuracy']*100:.1f}%)\")\n",
    "print(f\"Average Similarity: {performance['avg_similarity']:.3f}\")\n",
    "print(f\"No Match Rate: {performance['no_match_rate']:.3f} ({performance['no_match_rate']*100:.1f}%)\")\n",
    "\n",
    "print(\"\\n🎯 Confidence Distribution:\")\n",
    "for confidence, count in performance['confidence_distribution'].items():\n",
    "    print(f\"  {confidence}: {count} queries\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Model Analysis and Visualization\n",
    "\n",
    "Let's analyze our model's performance and create some visualizations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze similarity score distribution\n",
    "all_similarities = []\n",
    "sample_queries = df['question'].sample(100, random_state=42)\n",
    "\n",
    "for query in sample_queries:\n",
    "    matches = faq_model.find_best_match(query, top_k=1, min_similarity=0.0)\n",
    "    if matches:\n",
    "        all_similarities.append(matches[0]['similarity'])\n",
    "\n",
    "# Create visualizations\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "fig.suptitle('SATIM FAQ Model Analysis', fontsize=16, fontweight='bold')\n",
    "\n",
    "# Similarity distribution\n",
    "axes[0, 0].hist(all_similarities, bins=20, alpha=0.7, color='skyblue', edgecolor='black')\n",
    "axes[0, 0].set_title('Distribution of Similarity Scores')\n",
    "axes[0, 0].set_xlabel('Similarity Score')\n",
    "axes[0, 0].set_ylabel('Frequency')\n",
    "axes[0, 0].grid(True, alpha=0.3)\n",
    "\n",
    "# Category distribution in dataset\n",
    "category_counts = df['category'].value_counts()\n",
    "colors = plt.cm.Set3(np.linspace(0, 1, len(category_counts)))\n",
    "axes[0, 1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', colors=colors)\n",
    "axes[0, 1].set_title('FAQ Categories Distribution')\n",
    "\n",
    "# Question length distribution\n",
    "question_lengths = df['question'].str.len()\n",
    "axes[1, 0].hist(question_lengths, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')\n",
    "axes[1, 0].set_title('Question Length Distribution')\n",
    "axes[1, 0].set_xlabel('Characters')\n",
    "axes[1, 0].set_ylabel('Frequency')\n",
    "axes[1, 0].grid(True, alpha=0.3)\n",
    "\n",
    "# Performance metrics visualization\n",
    "metrics = ['Top-1 Acc', 'Top-3 Acc', 'Avg Similarity', 'Coverage']\n",
    "values = [\n",
    "    performance['top1_accuracy'], \n",
    "    performance['top3_accuracy'], \n",
    "    performance['avg_similarity'],\n",
    "    1 - performance['no_match_rate']\n",
    "]\n",
    "\n",
    "bars = axes[1, 1].bar(metrics, values, alpha=0.7, color=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99'])\n",
    "axes[1, 1].set_title('Model Performance Metrics')\n",
    "axes[1, 1].set_ylabel('Score')\n",
    "axes[1, 1].set_ylim(0, 1)\n",
    "axes[1, 1].grid(True, alpha=0.3)\n",
    "\n",
    "# Add value labels on bars\n",
    "for bar, value in zip(bars, values):\n",
    "    height = bar.get_height()\n",
    "    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.01,\n",
    "                    f'{value:.3f}', ha='center', va='bottom')\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Print detailed statistics\n",
    "print(\"\\n📈 Detailed Statistics:\")\n",
    "print(f\"Total FAQs: {len(df)}\")\n",
    "print(f\"Average question length: {question_lengths.mean():.1f} characters\")\n",
    "print(f\"Median question length: {question_lengths.median():.1f} characters\")\n",
    "print(f\"Average similarity score: {np.mean(all_similarities):.3f}\")\n",
    "print(f\"Standard deviation of similarities: {np.std(all_similarities):.3f}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Category-Based Analysis\n",
    "\n",
    "Let's analyze model performance by FAQ category."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze performance by category\n",
    "def analyze_by_category(model, df):\n",
    "    \"\"\"\n",
    "    Analyze model performance by FAQ category\n",
    "    \"\"\"\n",
    "    category_performance = {}\n",
    "    \n",
    "    for category in df['category'].unique():\n",
    "        category_df = df[df['category'] == category]\n",
    "        category_queries = category_df['question'].sample(min(10, len(category_df)), random_state=42)\n",
    "        \n",
    "        similarities = []\n",
    "        correct_matches = 0\n",
    "        total_queries = len(category_queries)\n",
    "        \n",
    "        for query in category_queries:\n",
    "            matches = model.find_best_match(query, top_k=1, min_similarity=0.0)\n",
    "            if matches:\n",
    "                similarities.append(matches[0]['similarity'])\n",
    "                if matches[0]['category'] == category:\n",
    "                    correct_matches += 1\n",
    "        \n",
    "        category_performance[category] = {\n",
    "            'avg_similarity': np.mean(similarities) if similarities else 0,\n",
    "            'category_accuracy': correct_matches / total_queries if total_queries > 0 else 0,\n",
    "            'total_faqs': len(category_df),\n",
    "            'queries_tested': total_queries\n",
    "        }\n",
    "    \n",
    "    return category_performance\n",
    "\n",
    "# Perform category analysis\n",
    "category_perf = analyze_by_category(faq_model, df)\n",
    "\n",
    "print(\"\\n📊 Performance by Category:\")\n",
    "print(\"=\" * 70)\n",
    "\n",
    "for category, metrics in category_perf.items():\n",
    "    print(f\"\\n📁 {category}:\")\n",
    "    print(f\"  Total FAQs: {metrics['total_faqs']}\")\n",
    "    print(f\"  Average Similarity: {metrics['avg_similarity']:.3f}\")\n",
    "    print(f\"  Category Accuracy: {metrics['category_accuracy']:.3f} ({metrics['category_accuracy']*100:.1f}%)\")\n",
    "    print(f\"  Queries Tested: {metrics['queries_tested']}\")\n",
    "\n",
    "# Visualize category performance\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))\n",
    "\n",
    "categories = list(category_perf.keys())\n",
    "similarities = [category_perf[cat]['avg_similarity'] for cat in categories]\n",
    "accuracies = [category_perf[cat]['category_accuracy'] for cat in categories]\n",
    "\n",
    "# Average similarity by category\n",
    "bars1 = ax1.bar(categories, similarities, alpha=0.7, color='lightblue')\n",
    "ax1.set_title('Average Similarity by Category')\n",
    "ax1.set_ylabel('Average Similarity')\n",
    "ax1.tick_params(axis='x', rotation=45)\n",
    "ax1.grid(True, alpha=0.3)\n",
    "\n",
    "# Add value labels\n",
    "for bar, value in zip(bars1, similarities):\n",
    "    height = bar.get_height()\n",
    "    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,\n",
    "            f'{value:.3f}', ha='center', va='bottom')\n",
    "\n",
    "# Category accuracy\n",
    "bars2 = ax2.bar(categories, accuracies, alpha=0.7, color='lightcoral')\n",